<a href="https://colab.research.google.com/github/Priyo-prog/Time-series-analysis/blob/main/Intoduction/Time_series_white_noise_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# White Noise Data 

White noise is a special type of time-series where data doesn't follow a pattern.

## Import packages and libraries

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.graphics.tsaplots as sgt
import statsmodels.tsa.stattools as sts
from statsmodels.tsa.seasonal import seasonal_decompose
import seaborn as sns
sns.set()

## Import the data